In [1]:
#notebook to calculate accuracy of the models and compare to benchmarks community votes and betting odds
#import necessary packages
import pandas as pd
import numpy as np

In [5]:
#function to calculate rps value
def rps(predictions, observed):
    """Vectorized version of Ranked Probability Score.
    A lower value is a better score.
    From: Colin Catlin, https://syllepsis.live/2022/01/22/ranked-probability-score-in-python/

    Args:
        predictions (pd.DataFrame): each column is an outcome category
            with values as the 0 to 1 probability of that category
        observed (pd.DataFrame): each column is an outcome category
            with values of 0 OR 1 with 1 being that category occurred
    """
    assert (
        predictions.shape == observed.shape
    ), "prediction and observed array shapes must match"
    ncat = predictions.shape[1] - 1
    return (
        np.sum(
            (np.cumsum(predictions, axis=1) - np.cumsum(observed, axis=1)) ** 2, axis=1
        ) / ncat
    )
predictions_df = pd.DataFrame({
    'H': [1, 0, 0,0.8, 0.5, 0.35, 0.6, 0.6, 0.6, 0.5],
    'D': [0, 0, 1,0.1, 0.25, 0.3, 0.3, 0.3, 0.1, 0.45],
    'A': [0, 1, 0,0.1, 0.25, 0.35, 0.1, 0.1, 0.3, 0.05],
    "model": np.tile(["a", "b"], 5),
    "match": np.repeat(np.arange(1, 6), 2),
    "outcome": np.repeat(["H", "H", "D", "H", "H"], 2),
})
expected_result = [0, 0.005, 0.025, 0.15625, 0.1225, 0.185, 0.085, 0.125, 0.12625, 0.1625]
predictions = predictions_df[["H", "D", "A"]]
observed = pd.get_dummies(predictions_df["outcome"]).reindex(
    columns=predictions.columns, fill_value=0
)
rps_result = rps(predictions, observed).round(5)

obs = ['H', 'H', 'H', 'H', 'D', 'D', 'H', 'H', 'H', 'H']
predictions['Result'] = obs
predictions['RPS'] = rps_result
predictions = predictions.rename(columns={'H':'Home win','D':'Draw','A':'Away win'})
predictions


#print(f"do results match expected: {all(expected_result == rps_result)}")

#print(np.mean(rps_result))

C:\Users\giuli\AppData\Local\Temp\ipykernel_1632\2609951829.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['Result'] = obs
C:\Users\giuli\AppData\Local\Temp\ipykernel_1632\2609951829.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions['RPS'] = rps_result


,Home win,Draw,Away win,Result,RPS
0,1.00,0.00,0.00,H,0.00000
1,0.00,0.00,1.00,H,1.00000
2,0.00,1.00,0.00,H,0.50000
3,0.80,0.10,0.10,H,0.02500
4,0.50,0.25,0.25,D,0.15625
5,0.35,0.30,0.35,D,0.12250
6,0.60,0.30,0.10,H,0.08500
7,0.60,0.30,0.10,H,0.08500
8,0.60,0.10,0.30,H,0.12500
9,0.50,0.45,0.05,H,0.12625


In [8]:
#calculate rps value for betting odds
PL_20_21 = pd.read_csv('Data/2020-21/Fixtures/E_20_21.csv')
PL_21_22 = pd.read_csv('Data/2021-22/Fixtures/E_21_22.csv')
#invert odds
PL_20_21['B365H'] = 1/PL_20_21['B365H']
PL_20_21['B365D'] = 1/PL_20_21['B365D']
PL_20_21['B365A'] = 1/PL_20_21['B365A']
PL_20_21['SUM'] = PL_20_21['B365H'] + PL_20_21['B365D'] + PL_20_21['B365A']
#calculate fair odds
PL_20_21['B365H'] = PL_20_21['B365H']/PL_20_21['SUM']
PL_20_21['B365D'] = PL_20_21['B365D']/PL_20_21['SUM']
PL_20_21['B365A'] = PL_20_21['B365A']/PL_20_21['SUM']
PL_21_22['B365H'] = 1/PL_21_22['B365H']
PL_21_22['B365D'] = 1/PL_21_22['B365D']
PL_21_22['B365A'] = 1/PL_21_22['B365A']
PL_21_22['SUM'] = PL_21_22['B365H'] + PL_21_22['B365D'] + PL_21_22['B365A']
PL_21_22['B365H'] = PL_21_22['B365H']/PL_21_22['SUM']
PL_21_22['B365D'] = PL_21_22['B365D']/PL_21_22['SUM']
PL_21_22['B365A'] = PL_21_22['B365A']/PL_21_22['SUM']
probabilities_20 = PL_20_21[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'FTR']]
probabilities_20 = probabilities_20.rename(columns={'B365H':'H','B365D':'D','B365A':'A'})
predictions_20 = probabilities_20[["H", "D", "A"]]
observed_20 = pd.get_dummies(probabilities_20["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)

probabilities_21 = PL_21_22[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'FTR']]
probabilities_21 = probabilities_21[-350:]
probabilities_21 = probabilities_21.rename(columns={'B365H':'H','B365D':'D','B365A':'A'})
predictions_21 = probabilities_21[["H", "D", "A"]]
observed_21 = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)



rps_result_21 = rps(predictions_21, observed_21).round(5)
#rps_result_20 = rps(predictions_20, observed_20).round(5)

rps_betting_odds = np.mean(rps_result_21)
print(rps_betting_odds)

AssertionError: prediction and observed array shapes must match

In [61]:
#calculate rps value for community votes
PL_21_22.replace(['Leeds', 'Leicester', 'Man City', 'Man United', 'Newcastle', 'Norwich', 'Sheff Utd', 'Spurs'], ['Leeds United', 'Leicester City', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
                  'Norwich City', 'Sheffield Utd', 'Tottenham'], inplace=True)
PL_21_22['game_id'] = PL_21_22['HomeTeam'] + PL_21_22['AwayTeam']
PL_21_22['HomeTeam'].drop_duplicates()
transfermarkt_probs_21 = pd.read_csv('Data/2021-22/Fixtures/tranfermarkt_21_probs.csv')
test = pd.merge(PL_21_22, transfermarkt_probs_21, on='game_id', how='left')

predictions = test[["H", "D", "A"]]
observed = pd.get_dummies(test["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result_community = rps(predictions, observed).round(5)

rps_community_votes = np.mean(rps_result_community)

print(rps_community_votes)

0.22040236842105265


In [58]:
#calculate rps value for rf classic model
probs_benchmark = pd.read_csv('Data/Total/probs_benchmark_info.csv')

predictions = probs_benchmark[["H", "D", "A"]]
observed = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result = rps(predictions, observed).round(5)

rps_rf_benchmark = np.mean(rps_result)
print(rps_rf_benchmark)

0.2156126842105263


In [59]:
#calculate rps score for rf improved model
probs_player_stats= pd.read_csv('Data/Total/probs_player_stats_info.csv')

predictions = probs_player_stats[["H", "D", "A"]]
observed = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result = rps(predictions, observed).round(5)

rps_rf_player_stats = np.mean(rps_result)
print(rps_rf_player_stats)

0.21135973684210527


In [60]:
#display rps values
print('RPS score random forest benchmark:', rps_rf_benchmark)
print('RPS score random forest incl player stats:', rps_rf_player_stats)
print('RPS score community votes:', rps_community_votes)
print('RPS score betting odds:', rps_betting_odds)

RPS score random forest benchmark: 0.2156126842105263
RPS score random forest incl player stats: 0.21135973684210527
RPS score community votes: 0.22040236842105265
RPS score betting odds: 0.19134634285714286


In [67]:
probs_player_stats= pd.read_csv('Data/Total/probs_player_stats_info.csv')

predictions = probs_player_stats[["H", "D", "A"]]
observed = pd.get_dummies(probabilities_21["FTR"]).reindex(
    columns=predictions.columns, fill_value=0)
rps_result = rps(predictions, observed).round(5)

probs_player_stats['rps'] = rps_result = rps(predictions, observed).round(3)
probs_player_stats

,Unnamed: 0,HomeTeam,AwayTeam,FTR,H,D,A,player_stats_vote,rps
0,0,Crystal Palace,Tottenham,H,0.637693,0.176003,0.186304,H,0.000
1,1,Arsenal,Norwich City,H,0.659592,0.209283,0.131125,H,0.000
2,2,Brentford,Brighton,A,0.169527,0.151281,0.679192,A,0.000
3,3,Leicester City,Manchester City,A,0.103872,0.156836,0.739292,A,0.000
4,4,Manchester Utd,Newcastle Utd,H,0.867363,0.068092,0.064545,H,0.000
...,...,...,...,...,...,...,...,...,...
345,345,Crystal Palace,Manchester Utd,H,0.270560,0.358476,0.370964,A,0.335
346,346,Leicester City,Southampton,H,0.621875,0.199979,0.178146,H,0.087
347,347,Liverpool,Wolves,H,0.579875,0.118616,0.301509,H,0.214
348,348,Manchester City,Aston Villa,H,0.672318,0.139366,0.188315,H,0.071
